In [0]:
# wigdet text, gdzie mozesz wpisac nazwe akcji
dbutils.widgets.text("ticker", "")
# get value from widget
ticker = dbutils.widgets.get("ticker")

In [0]:
from lib.api_extraction import downlaod_stock
from datetime import datetime, timedelta


end_date = datetime.today()
start_date = end_date - timedelta(days=40*365)


pdf = downlaod_stock(ticker, start_date, end_date)
pdf

In [0]:
pdf = pdf.reset_index() #pdf.reset_index(inplace=True)
pdf

In [0]:
# Create a Spark DataFrame from a pandas DataFrame using Arrow
new_columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df = spark.createDataFrame(pdf, new_columns)

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
#result_pdf = df.select("*").toPandas()

In [0]:
spark.read.table("databricksformula1.default.stocks").schema

In [0]:
# funkcje dodajaca metadane: ticker, created_timestamp, ingestion_date(yyyy-mm-dd)
import uuid
run_id = str(uuid.uuid4())
print(run_id)

In [0]:
from pyspark.sql.functions import lit

In [0]:
df = df.withColumn("ticker", lit(ticker))
df = df.withColumn("created_timestamp", lit(datetime.now()))
df = df.withColumn("ingestion_date", lit(datetime.now().strftime("%Y-%m-%d")))


In [0]:
display(df)

In [0]:
#modyfikacja na append, dodawac inkremental dane
df.write.mode("append").option("mergeSchema", "true").saveAsTable("databricksformula1.default.stocks")

In [0]:
%sql

SELECT ticker, count(*) FROM databricksformula1.default.stocks
GROUP BY ticker

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

In [0]:
stocks_schema = StructType([
    StructField("ingestion_date", StringType(), True),
    StructField("created_timestamp", TimestampType(), True),
    StructField("run_id", StringType(), True),
    StructField("ticker", StringType(), True),
    StructField("extraction_from", DateType(), True),
    StructField("extraction_to", DateType(), True),
    StructField("status", StringType(), True)
])

# empty table create in UC
spark.createDataFrame([], stocks_schema).write.mode("overwrite").saveAsTable("databricksformula1.default.stocks_metadata")

In [0]:
from datetime import datetime

In [0]:
df = spark.createDataFrame([(datetime.now().strftime("%Y-%m-%d"), datetime.now(), run_id, ticker, start_date, end_date, "Fail")], stocks_schema)

In [0]:
df.write.mode("append").saveAsTable("databricksformula1.default.stocks_metadata")